In [18]:
import shutil
shutil.unpack_archive('/content/AfricanElephantvsKangaroo.zip', 'FinalDS')

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import vgg16
import time

time_start = time.time()

# Define the VGG-16 model
class VGG16Binary(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG16Binary, self).__init__()
        self.vgg16 = vgg16(pretrained=True)
        self.vgg16.classifier[6] = nn.Linear(4096, num_classes)

    def forward(self, x):
        return self.vgg16(x)

# Load the dataset from the zip file
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = ImageFolder(root='/content/FinalDS/AfricanElephantvsKangaroo/Train', transform=data_transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VGG16Binary(num_classes=2).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_correct = 0
    total_samples = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Compute accuracy for this batch
        _, predicted_class = torch.max(outputs, 1)
        total_correct += (predicted_class == labels).sum().item()
        total_samples += labels.size(0)

    # Calculate accuracy for the epoch
    epoch_accuracy = 100 * total_correct / total_samples

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {loss.item():.4f} - Accuracy: {epoch_accuracy:.2f}%")


# End Time
time_end = time.time()

# Print time taken
print(f'Time taken: {time_end - time_start} seconds')


Epoch [1/5] - Loss: 2.9230 - Accuracy: 54.38%
Epoch [2/5] - Loss: 1.6450 - Accuracy: 41.25%
Epoch [3/5] - Loss: 0.8359 - Accuracy: 48.75%
Epoch [4/5] - Loss: 0.7663 - Accuracy: 41.25%
Epoch [5/5] - Loss: 0.7100 - Accuracy: 45.62%
Time taken: 17.533326625823975 seconds


In [22]:

test_dataset = ImageFolder(root='/content/FinalDS/AfricanElephantvsKangaroo/Test', transform=data_transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [23]:
# Assuming you have a test dataloader (test_dataloader) and a trained model (model)
model.eval()  # Set the model to evaluation mode

total_correct = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted_class = torch.max(outputs, 1)
        total_correct += (predicted_class == labels).sum().item()
        total_samples += labels.size(0)

test_accuracy = 100 * total_correct / total_samples
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 50.00%
